# Paper 05 — GSE249279 Visium QC

Loads processed Visium `.h5ad` files and produces basic QC plots.

In [ ]:
import json
from pathlib import Path
import scanpy as sc
import matplotlib.pyplot as plt

processed_dir = Path("../data/processed/spatial")
summary = json.loads((processed_dir / "load_summary.json").read_text())
len(summary), summary[0]


In [ ]:
ads = {}
for rec in summary:
    ads[rec["sample_name"]] = sc.read_h5ad(rec["out_h5ad"])
list(ads)[:5]


In [ ]:
spots = {k: v.n_obs for k, v in ads.items()}
plt.figure()
plt.bar(range(len(spots)), list(spots.values()))
plt.xticks(range(len(spots)), list(spots.keys()), rotation=90)
plt.ylabel("Spots")
plt.title("Spots per tissue section (GSE249279)")
plt.tight_layout()
plt.show()


In [ ]:
ad_all = sc.concat(list(ads.values()), label="section", keys=list(ads.keys()), join="outer", index_unique="-")
sc.pp.calculate_qc_metrics(ad_all, inplace=True)
sc.pl.violin(ad_all, keys=["n_genes_by_counts","total_counts"], groupby="section", rotation=90, show=True)


In [ ]:
name0 = next(iter(ads))
ad0 = ads[name0]
sc.pl.spatial(ad0, color=["total_counts","n_genes_by_counts"], img_key="hires", size=1.3, show=True)
